In [5]:
# project.ipynb

import sqlite3, pandas as pd, random
from faker import Faker
import matplotlib.pyplot as plt
import os

# paths
db_path = "data/Cloud_shop.sqlite"
os.makedirs("data", exist_ok=True)
os.makedirs("images", exist_ok=True)

# connect
conn = sqlite3.connect(db_path)
cur = conn.cursor()

# reset schema
cur.executescript("""
DROP TABLE IF EXISTS customers;
DROP TABLE IF EXISTS products;
DROP TABLE IF EXISTS orders;

CREATE TABLE customers(
  customer_id INTEGER PRIMARY KEY,
  name TEXT,
  region TEXT
);

CREATE TABLE products(
  product_id INTEGER PRIMARY KEY,
  product_name TEXT,
  category TEXT,
  unit_price REAL
);

CREATE TABLE orders(
  order_id INTEGER PRIMARY KEY,
  customer_id INTEGER,
  product_id INTEGER,
  order_date DATE,
  quantity INTEGER,
  status TEXT,
  FOREIGN KEY(customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY(product_id) REFERENCES products(product_id)
);
""")

# fake data
fake = Faker()
regions = ["APAC", "NA", "EU", "LATAM"]
categories = ["Electronics", "Clothing", "Books", "Home"]

for i in range(200):
    cur.execute("INSERT INTO customers VALUES (?, ?, ?)",
                (i+1, fake.name(), random.choice(regions)))

for i in range(50):
    cur.execute("INSERT INTO products VALUES (?, ?, ?, ?)",
                (i+1, fake.word().title(), random.choice(categories),
                 round(random.uniform(10,500), 2)))

for i in range(1000):
    cur.execute("INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?)",
                (i+1, random.randint(1,200), random.randint(1,50),
                 fake.date_between(start_date="-1y", end_date="today"),
                 random.randint(1,5),
                 random.choice(["Completed","Cancelled"])))
conn.commit()

# --- Queries + Charts ---

# Monthly revenue
monthly = pd.read_sql("""
WITH li AS (
  SELECT o.order_date, o.quantity*p.unit_price AS rev
  FROM orders o JOIN products p ON o.product_id=p.product_id
  WHERE o.status='Completed'
)
SELECT strftime('%Y-%m', order_date) as month, SUM(rev) as revenue
FROM li GROUP BY month ORDER BY month;
""", conn)

plt.plot(monthly['month'], monthly['revenue'], marker='o')
plt.title("Monthly Revenue Trend")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig("images/monthly_revenue.png")
plt.close()

# Revenue by region
region = pd.read_sql("""
SELECT c.region, SUM(o.quantity*p.unit_price) AS revenue
FROM orders o 
JOIN customers c ON o.customer_id=c.customer_id
JOIN products p ON o.product_id=p.product_id
WHERE o.status='Completed'
GROUP BY c.region;
""", conn)

region.plot(kind="bar", x="region", y="revenue", legend=False)
plt.title("Revenue by Region")
plt.ylabel("Revenue")
plt.savefig("images/revenue_by_region.png")
plt.close()

# Top products
top_products = pd.read_sql("""
WITH ranked AS (
  SELECT p.product_name, SUM(o.quantity*p.unit_price) AS revenue,
         RANK() OVER(ORDER BY SUM(o.quantity*p.unit_price) DESC) AS rnk
  FROM orders o JOIN products p ON o.product_id=p.product_id
  WHERE o.status='Completed'
  GROUP BY product_name
)
SELECT * FROM ranked WHERE rnk <= 10;
""", conn)

top_products.plot(kind="bar", x="product_name", y="revenue", legend=False)
plt.title("Top 10 Products by Revenue")
plt.ylabel("Revenue")
plt.xticks(rotation=60, ha="right")
plt.tight_layout()
plt.savefig("images/top_products.png")
plt.close()

print("✅ Project complete! Charts saved in /images")


/tmp/ipykernel_30005/2166374140.py:63: DeprecationWarning: The default date adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cur.execute("INSERT INTO orders VALUES (?, ?, ?, ?, ?, ?)",


✅ Project complete! Charts saved in /images
